# Project Title

## Topic
*What problem are you (or your stakeholder) trying to address?*


## Project Question
*What specific question are you seeking to answer with this project?*
*This is not the same as the questions you ask to limit the scope of the project.*

## What would an answer look like?
*What is your hypothesized answer to your question?*


## Data Sources
*What 3 data sources have you identified for this project?*
*How are you going to relate these datasets?*
*How will you use this data to answer your project question?*

In [ ]:
# Start your code here